In [1]:
import pandas as pd
from mtl_data_to_text import get_justification_statement
from evaluate import load
import torch

/home/pk2743/just-verdict/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-14 00:47:20.043462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Source #1: REBEL + Ablations

In [31]:
data_df = pd.read_pickle('./data/data_verdict.pkl')

In [32]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,random_claim_evidence_1,random_entailment_scores_1,random_verdict,clean_verdict,verdict,verdict_actual
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...","{'(A baby died at an unnamed medical facility,...","[[0.6385632, -0.5613695, -0.52640414], [0.6686...",-1,-1,-1,0
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...","{'(Shawnee County, located in the administrati...","[[0.6175279, -0.7271561, -0.2998387], [-0.5644...",1,-1,-1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...","{'(banned pork from school canteens, country, ...","[[0.679509, -0.6404532, -0.35789278], [0.69187...",-1,-1,-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...","{'(Coronavirus, country, Canada)': ['(opioid, ...","[[0.69448966, -0.6130783, -0.37658876], [0.686...",-1,-1,-1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...","{'(Wayne National Forest, instance of, wildlif...","[[0.6313532, -0.7029955, -0.3273997], [-0.2482...",0,-1,-1,1


In [33]:
def get_justification(row, cleaned=True, random=False):
    if random:
        claim_evidences = row['random_claim_evidence_1']
        entailment_scores = row['random_entailment_scores_1']
    else:
        if cleaned:
            claim_evidences = row['cleaned_claim_evidence_1']
            entailment_scores = row['cleaned_entailment_scores_1']
        else:
            claim_evidences = row['claim_evidence_1']
            entailment_scores = row['entailment_scores_1']

    claims = list(claim_evidences.keys())

    index_values = [-1, 1, 0]

    justifications = []

    for i in range(len(claims)):
        which_verdict = entailment_scores[i].argmax()
        evidence = claim_evidences[claims[i]][0]
        claim = claims[i]
  
        justification = get_justification_statement(claim, evidence, index_values[which_verdict])

        justifications.append(justification)

    justification_mtl = ' '.join(justifications)
    
    if random:
        row['random_justification_mtl'] = justification_mtl
    else:
        if cleaned:
            row['cleaned_justification_mtl'] = justification_mtl
        else:
            row['justification_mtl'] = justification_mtl

    if row.name%100==0:
        print(row.name)
    
    return row


In [5]:
data_df = data_df.apply(lambda row : get_justification(row, cleaned=False), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [34]:
data_df = data_df.apply(lambda row : get_justification(row, cleaned=True, random=True), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [10]:
data_df = data_df.apply(lambda row : get_justification(row, cleaned=True), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [2]:
data_df = pd.read_pickle('./data/data_justifications.pkl')

In [3]:
data_df

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,verdict,verdict_actual,clean_verdict,justification_mtl,cleaned_justification_mtl,random_justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...",-1,0,-1,A baby died at an unnamed medical facility bec...,A baby died at an unnamed medical facility bec...,The reason a baby died at an unnamed medical f...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...",-1,1,-1,Bat from Shawnee County is not caused by rabie...,Topeka is not located in Shawnee County. Bat f...,KSNT is located in Topeka and not in Shawnee C...
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...",-1,-1,-1,Germany banned pork from school canteen becaus...,Germany banned pork from school canteen becaus...,Pork from school canteen is banned in Germany ...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...",-1,1,-1,Coronavirus is found in Canada and is unrelate...,Coronavirus is found in Canada and is unrelate...,"The Coronavirus is not found in Canada, but be..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...",-1,1,-1,"The Wayne National Forest is not a forest, bec...",The Wayne National Forest is not an instance o...,The Wayne National Forest is not an instance o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,3295,Christmas miracle for Florida dog whose heart ...,NaN,A Florida man was hoping for a Christmas mirac...,Associated Press,The heart of Gerald Ford’s 1-year-old French b...,https://www.gainesville.com/news/20200102/holi...,true,"Animals, Health, Flo

In [4]:
print(data_df.iloc[5]['claim'])
print(data_df.iloc[5]['explanation'])
print(data_df.iloc[5]['label'])
print(data_df.iloc[5]['cleaned_justification_mtl'])
print(data_df.iloc[5]['random_justification_mtl'])
print(data_df.iloc[5]['justification_mtl'])

School closures will have little impact on COVID-19 control, review finds.
School closures do not tend to help contain the spread of infections during outbreaks of disease such as COVID-19, but will have a big impact on how societies restart after lockdown, scientists said on Monday.
true
The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.
Robert Dingwall, whose field of study is sociology, is the subject of the COVID-19 control. Great Ormond Street Institute of Child Health is run by UCL and COVID-19 does not have effect on control.
The SARS-COV2 coronavirus caused school closures as part of COVID-19 control. The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.


In [47]:
data_df.to_pickle('./data/data_justifications.pkl')

In [5]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [48]:
data_df = pd.read_pickle('./data/data_justifications.pkl')

In [8]:
bertscore = load("bertscore")
rouge = load('rouge')
sacrebleu = load('sacrebleu')

def get_metrics(predictions, references, references_sb):
    results = {}    
    results["bertscore"] = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    results["rouge"] = rouge.compute(predictions=predictions, references=references)
    results["sacrebleu"] = sacrebleu.compute(predictions=predictions, references=references_sb)

    return results

In [9]:
random_justifications_mtl = data_df['random_justification_mtl'].tolist()
cleaned_justifications_mtl = data_df['cleaned_justification_mtl'].tolist()
justifications_mtl = data_df['justification_mtl'].tolist()
references = data_df['explanation'].tolist()
references_sb = [[reference] for reference in references]

In [10]:
random_results_mtl = get_metrics(random_justifications_mtl, references, references_sb)
print(random_results_mtl)
cleaned_results_mtl = get_metrics(cleaned_justifications_mtl, references, references_sb)
print(cleaned_results_mtl)
results_mtl = get_metrics(justifications_mtl, references, references_sb)
print(results_mtl)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 151kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 1.11MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 79.8MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 33.6MB/s]
model.safetensors: 100%|██████████| 268M/268M [00:00<00:00, 548MB/s] 


{'bertscore': {'precision': [0.7218058705329895, 0.7292712926864624, 0.7730293869972229, 0.7945643067359924, 0.7880454063415527, 0.7317682504653931, 0.7281996011734009, 0.7897610664367676, 0.7899374961853027, 0.6999733448028564, 0.7672601938247681, 0.8063132166862488, 0.834967851638794, 0.698204517364502, 0.6471977233886719, 0.5765023827552795, 0.7814615964889526, 0.7820117473602295, 0.7163269519805908, 0.8903895020484924, 0.7565922737121582, 0.7179648876190186, 0.8174636960029602, 0.8266541957855225, 0.6779496669769287, 0.7245855331420898, 0.791122317314148, 0.7253924012184143, 0.8255966305732727, 0.7543340921401978, 0.7472696304321289, 0.8130173087120056, 0.8102957010269165, 0.7364726066589355, 0.7827898859977722, 0.782317042350769, 0.754811704158783, 0.688288152217865, 0.778058648109436, 0.7532660365104675, 0.7789045572280884, 0.7657461762428284, 0.6881116628646851, 0.7609481811523438, 0.7065434455871582, 0.7939413785934448, 0.8184568285942078, 0.7268756031990051, 0.7340490818023682

In [11]:
cleaned_mtl_precision = sum(cleaned_results_mtl['bertscore']['precision']) / len(cleaned_results_mtl['bertscore']['precision'])
cleaned_mtl_recall = sum(cleaned_results_mtl['bertscore']['recall']) / len(cleaned_results_mtl['bertscore']['recall'])
cleaned_mtl_f1score = sum(cleaned_results_mtl['bertscore']['f1']) / len(cleaned_results_mtl['bertscore']['f1'])

print('With Redundancy Removal')
print(f"BERTScore Precision: {cleaned_mtl_precision}, Recall: {cleaned_mtl_recall}, F1-score: {cleaned_mtl_f1score}")

With Redundancy Removal
BERTScore Precision: 0.7698669620769837, Recall: 0.7149430647322249, F1-score: 0.7400893391296876


In [12]:
mtl_precision = sum(results_mtl['bertscore']['precision']) / len(results_mtl['bertscore']['precision'])
mtl_recall = sum(results_mtl['bertscore']['recall']) / len(results_mtl['bertscore']['recall'])
mtl_f1score = sum(results_mtl['bertscore']['f1']) / len(results_mtl['bertscore']['f1'])

print('Without Redundancy Removal')
print(f"BERTScore Precision: {mtl_precision}, Recall: {mtl_recall}, F1-score: {mtl_f1score}")

Without Redundancy Removal
BERTScore Precision: 0.7594125878558073, Recall: 0.7160805000955479, F1-score: 0.7358768231328121


In [13]:
random_mtl_precision = sum(random_results_mtl['bertscore']['precision']) / len(random_results_mtl['bertscore']['precision'])
random_mtl_recall = sum(random_results_mtl['bertscore']['recall']) / len(random_results_mtl['bertscore']['recall'])
random_mtl_f1score = sum(random_results_mtl['bertscore']['f1']) / len(random_results_mtl['bertscore']['f1'])

print('With Random Evidence Retrieval')
print(f"BERTScore Precision: {random_mtl_precision}, Recall: {random_mtl_recall}, F1-score: {random_mtl_f1score}")

With Random Evidence Retrieval
BERTScore Precision: 0.7605961342668844, Recall: 0.7236838507029983, F1-score: 0.7405239717023019


In [15]:
print(cleaned_results_mtl['rouge'])
print(results_mtl['rouge'])
print(random_results_mtl['rouge'])

{'rouge1': 0.1852365614516192, 'rouge2': 0.04423259214688255, 'rougeL': 0.1383762726891537, 'rougeLsum': 0.13835729220665433}
{'rouge1': 0.18598747650926112, 'rouge2': 0.04174691768922138, 'rougeL': 0.14016746568197114, 'rougeLsum': 0.14018280050294787}
{'rouge1': 0.18712004578112668, 'rouge2': 0.04110963199581587, 'rougeL': 0.1346756159316954, 'rougeLsum': 0.13478027813597376}


In [16]:
print(cleaned_results_mtl['sacrebleu'])
print(results_mtl['sacrebleu'])
print(random_results_mtl['sacrebleu'])

{'score': 0.6131975872914872, 'counts': [22547, 4054, 1153, 420], 'totals': [76698, 74246, 71794, 69342], 'precisions': [29.39711596130277, 5.460226813565714, 1.6059837869459843, 0.6056935190793459], 'bp': 0.17346947509517147, 'sys_len': 76698, 'ref_len': 211054}
{'score': 1.1458607457315961, 'counts': [28186, 4965, 1374, 489], 'totals': [107583, 105131, 102679, 100227], 'precisions': [26.19930658189491, 4.722679323891145, 1.3381509364134827, 0.48789248406118113], 'bp': 0.3822125743176673, 'sys_len': 107583, 'ref_len': 211054}
{'score': 0.7319894130561173, 'counts': [24314, 4205, 1116, 373], 'totals': [86574, 84122, 81670, 79218], 'precisions': [28.08464435049784, 4.998692375359597, 1.366474837761724, 0.4708525840086849], 'bp': 0.2374388694306979, 'sys_len': 86574, 'ref_len': 211054}


In [17]:
predictions = ["My name is Prahlad, I study at Columbia"]
references = ["My university is Columbia, I am Prahlad"]
results = sacrebleu.compute(predictions=predictions, references=references, tokenize='none', lowercase=True, use_effective_order=True)
print(results)

{'score': 7.267884212102741, 'counts': [3, 0, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [37.5, 7.142857142857143, 4.166666666666667, 2.5], 'bp': 1.0, 'sys_len': 8, 'ref_len': 7}


In [18]:
def get_stats(data_df, columns):

    for column in columns:
        data_df[f'{column}_tokens'] = data_df[column].apply(lambda n: len(str(n).split()))

get_stats(data_df, columns=['claim', 'main_text', 'explanation'])

In [19]:
small_just_df = data_df[(data_df['main_text_tokens'] < 100) & (data_df['main_text_tokens'] > 10) ]

In [20]:
small_just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,entailment_scores_1,verdict,verdict_actual,clean_verdict,justification_mtl,cleaned_justification_mtl,random_justification_mtl,claim_tokens,main_text_tokens,explanation_tokens
36,5583,Alabama: Case of meningococcal disease in Jack...,NaN,The Alabama Department of Public Health’s Immu...,,"In a news release Tuesday, the department said...",,true,"Immunizations, Meningitis, Health, Alabama, Pu...","[{'head': 'Alabama', 'type': 'contains adminis...",...,"[[-0.77345294, -0.0044897804, 0.6338379], [-0....",1,1,1,"Scottsboro is located in Jackson County, Alaba...","Scottsboro is located in Jackson County, Alaba...",Jackson County is located in the Administrativ...,8,90,34
121,40135,Little Lauren Renee Pingel was in a car accid...,"March 17, 2015",Pray for 3-year-old Lauren Renee Pingel in Ama...,Rich Buhler & Staff,This story is true… and the latest news is tha...,https://twitter.com/intent/tweet?text=Pray+for...,true,Prayers,"[{'head': 'Little Lauren Renee Pingel', 'type'...",...,"[[-0.32555494, -0.60986596, 0.7225494], [-0.30...",1,1,1,Little Lauren Renee Pingel was born on Decembe...,"Lauren Renee Pingel was born in Amarillo, Texa...","Lauren Renee Pingel was born in Amarillo, Texa...",32,69,9
189,7264,Zambia to vaccinate 1 million against cholera ...,NaN,Zambia is set to vaccinate one million people ...,,The World Health Organization says doses of th...,,true,"Health, Cholera, Zambia, Africa, Southern Afri...","[{'head': 'outbreak', 'type': 'has cause', 'ta...",...,"[[-0.6810705, -0.19560058, 0.7056086], [0.6290...",-1,1,-1,An outbreak of cholera is unrelated to acute d...,An outbreak of cholera is unrelated to acute d...,"Cholera is the cause of an outbreak in Zambia,...",9,93,18
190,5834,State to labs: report all results of tests for...,NaN,Louisiana’s Department of Health says laborato...,,Louisiana has very high rates of all three dis...,http://ldh.la.gov/index.cfm/newsroom/detail/5162,true,"Health, Hepatitis, Syphilis, Louisiana, Public...","[{'head': 'tests', 'type': 'facet of', 'tail':...",...,"[[-0.8129388, 0.15043166, 0.56258404], [0.0264...",1,1,1,The CDC is unrelated to Centers for Disease Co...,The CDC is unrelated to Centers for Disease Co...,Louisiana has tests for diseases unrelated to ...,11,96,27
237,4952,Health care worker in Grant County has mumps.,NaN,The Grant County Health District has issued an...,,The health district says a vaccinated health c...,http://www.kxly.com/,true,"Health, Moses Lake, Mumps",[{'head': 'Health care worker in Grant County'...,...,"[[0.59940296, -0.771737, -0.21245721], [-0.610...",-1,1,1,Samaritan Healthcare is located in Moses Lake ...,Samaritan Healthcare is located in Moses Lake ...,KXLY-TV is located in Spokane and not in Grant...,8,84,20


In [24]:
which_just = small_just_df.iloc[34]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['cleaned_justification_mtl'])
print(which_just['justification_mtl'])
print(which_just['random_justification_mtl'])

Ohio officials say tick-borne Lyme Disease still a threat.
The state agency reports that 241 cases of Lyme disease have been reported in 60 of Ohio’s 88 counties this year. The number of cases has risen steadily the last five years, with 160 reported last year after only 93 in 2013. Lyme disease is passed to humans by deer ticks and can cause flu-like illnesses, muscle pain and headaches. A rash shaped like a bull’s-eye is often seen around tick bites. The Ohio Department of Natural Resources recommends people walk in the middle of trails and tuck in clothing to prevent bites.
The Ohio Department of Health says hunters and those trekking in wooded areas should use bug spray and take other precautions even in cold weather to prevent being infected with tick-borne Lyme disease.
true
[{'head': 'Lyme Disease', 'type': 'has cause', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'head': 'Lyme Disease', 'type': 'subclass of', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'hea

In [25]:
which_just = small_just_df.loc[189]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['cleaned_justification_mtl'])
print(which_just['justification_mtl'])
print(which_just['random_justification_mtl'])

Zambia to vaccinate 1 million against cholera amid outbreak.
The World Health Organization says doses of the cholera vaccine have been delivered to the impoverished southern African nation as fears spread in Lusaka. Zambia’s health ministry says 58 of the 63 countrywide deaths in the cholera outbreak that began in early October have occurred in the capital. Of the 2,672 cases across the country, 2,558 have been in Lusaka. Cholera is an acute diarrheal and bacterial infection caused by ingestion of contaminated food or water. Just hours after being infected, vomiting and diarrhea cause severe dehydration that can kill without rapid intervention.
Zambia is set to vaccinate one million people in its capital against cholera as a deadly outbreak grows.
true
[{'head': 'outbreak', 'type': 'has cause', 'tail': 'cholera', 'meta': {'spans': [[0, 128]]}}, {'head': 'outbreak', 'type': 'country', 'tail': 'Zambia', 'meta': {'spans': [[0, 128]]}}, {'head': 'vaccine', 'type': 'country', 'tail': 'Zambi

# Data Source #2: FRED

In [ ]:
data_df = pd.read_pickle('./data/fred/data_verdict.pkl')

In [ ]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,random_claim_evidence_1,random_entailment_scores_1,random_verdict,clean_verdict,verdict,verdict_actual
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...","{'(A baby died at an unnamed medical facility,...","[[0.6385632, -0.5613695, -0.52640414], [0.6686...",-1,-1,-1,0
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...","{'(Shawnee County, located in the administrati...","[[0.6175279, -0.7271561, -0.2998387], [-0.5644...",1,-1,-1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...","{'(banned pork from school canteens, country, ...","[[0.679509, -0.6404532, -0.35789278], [0.69187...",-1,-1,-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...","{'(Coronavirus, country, Canada)': ['(opioid, ...","[[0.69448966, -0.6130783, -0.37658876], [0.686...",-1,-1,-1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...","{'(Wayne National Forest, instance of, wildlif...","[[0.6313532, -0.7029955, -0.3273997], [-0.2482...",0,-1,-1,1


In [ ]:
def get_justification(row):
    claim_evidences = row['fred_claim_evidence_1']
    entailment_scores = row['fred_entailment_scores_1']

    claims = list(claim_evidences.keys())

    index_values = [-1, 1, 0]

    justifications = []

    for i in range(len(claims)):
        which_verdict = entailment_scores[i].argmax()
        evidence = claim_evidences[claims[i]][0]
        claim = claims[i]
  
        justification = get_justification_statement(claim, evidence, index_values[which_verdict])

        justifications.append(justification)

    justification_mtl = ' '.join(justifications)
    
    row['justification_mtl'] = justification_mtl

    if row.name%100==0:
        print(row.name)
    
    return row


In [ ]:
data_df = data_df.apply(lambda row : get_justification(row), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [ ]:
data_df

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,verdict,verdict_actual,clean_verdict,justification_mtl,cleaned_justification_mtl,random_justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...",-1,0,-1,A baby died at an unnamed medical facility bec...,A baby died at an unnamed medical facility bec...,The reason a baby died at an unnamed medical f...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...",-1,1,-1,Bat from Shawnee County is not caused by rabie...,Topeka is not located in Shawnee County. Bat f...,KSNT is located in Topeka and not in Shawnee C...
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...",-1,-1,-1,Germany banned pork from school canteen becaus...,Germany banned pork from school canteen becaus...,Pork from school canteen is banned in Germany ...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...",-1,1,-1,Coronavirus is found in Canada and is unrelate...,Coronavirus is found in Canada and is unrelate...,"The Coronavirus is not found in Canada, but be..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...",-1,1,-1,"The Wayne National Forest is not a forest, bec...",The Wayne National Forest is not an instance o...,The Wayne National Forest is not an instance o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,3295,Christmas miracle for Florida dog whose heart ...,NaN,A Florida man was hoping for a Christmas mirac...,Associated Press,The heart of Gerald Ford’s 1-year-old French b...,https://www.gainesville.com/news/20200102/holi...,true,"Animals, Health, Flo

In [ ]:
print(data_df.iloc[5]['claim'])
print(data_df.iloc[5]['explanation'])
print(data_df.iloc[5]['label'])
print(data_df.iloc[5]['justification_mtl'])

School closures will have little impact on COVID-19 control, review finds.
School closures do not tend to help contain the spread of infections during outbreaks of disease such as COVID-19, but will have a big impact on how societies restart after lockdown, scientists said on Monday.
true
The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.
Robert Dingwall, whose field of study is sociology, is the subject of the COVID-19 control. Great Ormond Street Institute of Child Health is run by UCL and COVID-19 does not have effect on control.
The SARS-COV2 coronavirus caused school closures as part of COVID-19 control. The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.


In [ ]:
data_df.to_pickle('./data/fred/data_justifications.pkl')

In [ ]:
data_df = pd.read_pickle('./data/fred/data_justifications.pkl')

In [ ]:
bertscore = load("bertscore")
rouge = load('rouge')
sacrebleu = load('sacrebleu')

def get_metrics(predictions, references, references_sb):
    results = {}    
    results["bertscore"] = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    results["rouge"] = rouge.compute(predictions=predictions, references=references)
    results["sacrebleu"] = sacrebleu.compute(predictions=predictions, references=references_sb)

    return results

In [ ]:
justifications_mtl = data_df['justification_mtl'].tolist()
references = data_df['explanation'].tolist()
references_sb = [[reference] for reference in references]

In [ ]:
results_mtl = get_metrics(justifications_mtl, references, references_sb)
print(results_mtl)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 151kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 1.11MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 79.8MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 33.6MB/s]
model.safetensors: 100%|██████████| 268M/268M [00:00<00:00, 548MB/s] 


{'bertscore': {'precision': [0.7218058705329895, 0.7292712926864624, 0.7730293869972229, 0.7945643067359924, 0.7880454063415527, 0.7317682504653931, 0.7281996011734009, 0.7897610664367676, 0.7899374961853027, 0.6999733448028564, 0.7672601938247681, 0.8063132166862488, 0.834967851638794, 0.698204517364502, 0.6471977233886719, 0.5765023827552795, 0.7814615964889526, 0.7820117473602295, 0.7163269519805908, 0.8903895020484924, 0.7565922737121582, 0.7179648876190186, 0.8174636960029602, 0.8266541957855225, 0.6779496669769287, 0.7245855331420898, 0.791122317314148, 0.7253924012184143, 0.8255966305732727, 0.7543340921401978, 0.7472696304321289, 0.8130173087120056, 0.8102957010269165, 0.7364726066589355, 0.7827898859977722, 0.782317042350769, 0.754811704158783, 0.688288152217865, 0.778058648109436, 0.7532660365104675, 0.7789045572280884, 0.7657461762428284, 0.6881116628646851, 0.7609481811523438, 0.7065434455871582, 0.7939413785934448, 0.8184568285942078, 0.7268756031990051, 0.7340490818023682

In [ ]:
mtl_precision = sum(results_mtl['bertscore']['precision']) / len(results_mtl['bertscore']['precision'])
mtl_recall = sum(results_mtl['bertscore']['recall']) / len(results_mtl['bertscore']['recall'])
mtl_f1score = sum(results_mtl['bertscore']['f1']) / len(results_mtl['bertscore']['f1'])

print(f"BERTScore Precision: {mtl_precision}, Recall: {mtl_recall}, F1-score: {mtl_f1score}")

Without Redundancy Removal
BERTScore Precision: 0.7594125878558073, Recall: 0.7160805000955479, F1-score: 0.7358768231328121


In [ ]:
print(results_mtl['rouge'])

{'rouge1': 0.1852365614516192, 'rouge2': 0.04423259214688255, 'rougeL': 0.1383762726891537, 'rougeLsum': 0.13835729220665433}
{'rouge1': 0.18598747650926112, 'rouge2': 0.04174691768922138, 'rougeL': 0.14016746568197114, 'rougeLsum': 0.14018280050294787}
{'rouge1': 0.18712004578112668, 'rouge2': 0.04110963199581587, 'rougeL': 0.1346756159316954, 'rougeLsum': 0.13478027813597376}


In [ ]:
print(results_mtl['sacrebleu'])

{'score': 0.6131975872914872, 'counts': [22547, 4054, 1153, 420], 'totals': [76698, 74246, 71794, 69342], 'precisions': [29.39711596130277, 5.460226813565714, 1.6059837869459843, 0.6056935190793459], 'bp': 0.17346947509517147, 'sys_len': 76698, 'ref_len': 211054}
{'score': 1.1458607457315961, 'counts': [28186, 4965, 1374, 489], 'totals': [107583, 105131, 102679, 100227], 'precisions': [26.19930658189491, 4.722679323891145, 1.3381509364134827, 0.48789248406118113], 'bp': 0.3822125743176673, 'sys_len': 107583, 'ref_len': 211054}
{'score': 0.7319894130561173, 'counts': [24314, 4205, 1116, 373], 'totals': [86574, 84122, 81670, 79218], 'precisions': [28.08464435049784, 4.998692375359597, 1.366474837761724, 0.4708525840086849], 'bp': 0.2374388694306979, 'sys_len': 86574, 'ref_len': 211054}


In [ ]:
predictions = ["My name is Prahlad, I study at Columbia"]
references = ["My university is Columbia, I am Prahlad"]
results = sacrebleu.compute(predictions=predictions, references=references, tokenize='none', lowercase=True, use_effective_order=True)
print(results)

{'score': 7.267884212102741, 'counts': [3, 0, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [37.5, 7.142857142857143, 4.166666666666667, 2.5], 'bp': 1.0, 'sys_len': 8, 'ref_len': 7}


In [ ]:
def get_stats(data_df, columns):

    for column in columns:
        data_df[f'{column}_tokens'] = data_df[column].apply(lambda n: len(str(n).split()))

get_stats(data_df, columns=['claim', 'main_text', 'explanation'])

In [ ]:
small_just_df = data_df[(data_df['main_text_tokens'] < 100) & (data_df['main_text_tokens'] > 10) ]

In [ ]:
small_just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,entailment_scores_1,verdict,verdict_actual,clean_verdict,justification_mtl,cleaned_justification_mtl,random_justification_mtl,claim_tokens,main_text_tokens,explanation_tokens
36,5583,Alabama: Case of meningococcal disease in Jack...,NaN,The Alabama Department of Public Health’s Immu...,,"In a news release Tuesday, the department said...",,true,"Immunizations, Meningitis, Health, Alabama, Pu...","[{'head': 'Alabama', 'type': 'contains adminis...",...,"[[-0.77345294, -0.0044897804, 0.6338379], [-0....",1,1,1,"Scottsboro is located in Jackson County, Alaba...","Scottsboro is located in Jackson County, Alaba...",Jackson County is located in the Administrativ...,8,90,34
121,40135,Little Lauren Renee Pingel was in a car accid...,"March 17, 2015",Pray for 3-year-old Lauren Renee Pingel in Ama...,Rich Buhler & Staff,This story is true… and the latest news is tha...,https://twitter.com/intent/tweet?text=Pray+for...,true,Prayers,"[{'head': 'Little Lauren Renee Pingel', 'type'...",...,"[[-0.32555494, -0.60986596, 0.7225494], [-0.30...",1,1,1,Little Lauren Renee Pingel was born on Decembe...,"Lauren Renee Pingel was born in Amarillo, Texa...","Lauren Renee Pingel was born in Amarillo, Texa...",32,69,9
189,7264,Zambia to vaccinate 1 million against cholera ...,NaN,Zambia is set to vaccinate one million people ...,,The World Health Organization says doses of th...,,true,"Health, Cholera, Zambia, Africa, Southern Afri...","[{'head': 'outbreak', 'type': 'has cause', 'ta...",...,"[[-0.6810705, -0.19560058, 0.7056086], [0.6290...",-1,1,-1,An outbreak of cholera is unrelated to acute d...,An outbreak of cholera is unrelated to acute d...,"Cholera is the cause of an outbreak in Zambia,...",9,93,18
190,5834,State to labs: report all results of tests for...,NaN,Louisiana’s Department of Health says laborato...,,Louisiana has very high rates of all three dis...,http://ldh.la.gov/index.cfm/newsroom/detail/5162,true,"Health, Hepatitis, Syphilis, Louisiana, Public...","[{'head': 'tests', 'type': 'facet of', 'tail':...",...,"[[-0.8129388, 0.15043166, 0.56258404], [0.0264...",1,1,1,The CDC is unrelated to Centers for Disease Co...,The CDC is unrelated to Centers for Disease Co...,Louisiana has tests for diseases unrelated to ...,11,96,27
237,4952,Health care worker in Grant County has mumps.,NaN,The Grant County Health District has issued an...,,The health district says a vaccinated health c...,http://www.kxly.com/,true,"Health, Moses Lake, Mumps",[{'head': 'Health care worker in Grant County'...,...,"[[0.59940296, -0.771737, -0.21245721], [-0.610...",-1,1,1,Samaritan Healthcare is located in Moses Lake ...,Samaritan Healthcare is located in Moses Lake ...,KXLY-TV is located in Spokane and not in Grant...,8,84,20


In [ ]:
which_just = small_just_df.iloc[34]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['justification_mtl'])

Ohio officials say tick-borne Lyme Disease still a threat.
The state agency reports that 241 cases of Lyme disease have been reported in 60 of Ohio’s 88 counties this year. The number of cases has risen steadily the last five years, with 160 reported last year after only 93 in 2013. Lyme disease is passed to humans by deer ticks and can cause flu-like illnesses, muscle pain and headaches. A rash shaped like a bull’s-eye is often seen around tick bites. The Ohio Department of Natural Resources recommends people walk in the middle of trails and tuck in clothing to prevent bites.
The Ohio Department of Health says hunters and those trekking in wooded areas should use bug spray and take other precautions even in cold weather to prevent being infected with tick-borne Lyme disease.
true
[{'head': 'Lyme Disease', 'type': 'has cause', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'head': 'Lyme Disease', 'type': 'subclass of', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'hea

In [ ]:
which_just = small_just_df.loc[189]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['justification_mtl'])

Zambia to vaccinate 1 million against cholera amid outbreak.
The World Health Organization says doses of the cholera vaccine have been delivered to the impoverished southern African nation as fears spread in Lusaka. Zambia’s health ministry says 58 of the 63 countrywide deaths in the cholera outbreak that began in early October have occurred in the capital. Of the 2,672 cases across the country, 2,558 have been in Lusaka. Cholera is an acute diarrheal and bacterial infection caused by ingestion of contaminated food or water. Just hours after being infected, vomiting and diarrhea cause severe dehydration that can kill without rapid intervention.
Zambia is set to vaccinate one million people in its capital against cholera as a deadly outbreak grows.
true
[{'head': 'outbreak', 'type': 'has cause', 'tail': 'cholera', 'meta': {'spans': [[0, 128]]}}, {'head': 'outbreak', 'type': 'country', 'tail': 'Zambia', 'meta': {'spans': [[0, 128]]}}, {'head': 'vaccine', 'type': 'country', 'tail': 'Zambi

# Data Source #3: SpaCy

In [2]:
data_df = pd.read_pickle('./data/spacy/data_verdict.pkl')

In [3]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,spacy_main_text_kg,spacy_claim_kg,spacy_claim_kg_size,spacy_main_text_kg_size,spacy_claim_triples,spacy_main_text_triples,spacy_claim_evidence_1,spacy_entailment_scores_1,verdict_actual,verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[{'head': '@FierceFemtivist', 'type': 'publish...","[{'head': 'baby', 'type': 'died', 'tail': 'fac...",2,54,"[(baby, died, facility), (nurse, died, child)]","[(@FierceFemtivist, published, series), (child...","{'(baby, died, facility)': ['(baby, reads, eve...","[[0.67679816, -0.60078514, -0.4254427], [0.677...",0,-1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'bat', 'type': 'reports', 'tail': 'C...","[{'head': 'Bat', 'type': 'tests', 'tail': 'Cou...",2,12,"[(Bat, tests, County), (Bat, tests, rabies)]","[(bat, reports, County), (Department, urging, ...","{'(Bat, tests, County)': ['(bat, reports, Coun...","[[-0.20893203, -0.7196208, 0.66218823], [0.569...",1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'cafes', 'type': 'reported', 'tail':...","[{'head': 'Germany', 'type': 'banned', 'tail':...",3,168,"[(Germany, banned, pork), (Germany, banned, ca...","[(cafes, reported, products), (GERMANY, made, ...","{'(Germany, banned, pork)': ['(frenzy, banned,...","[[-0.20289786, -0.7068694, 0.67761946], [0.638...",-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'government', 'type': 'urged', 'tail...","[{'head': 'Coronavirus', 'type': 'prompts', 't...",3,114,"[(Coronavirus, prompts, drugs), (Coronavirus, ...","[(government, urged, provinces), (government, ...","{'(Coronavirus, prompts, drugs)': ['(who, ramp...","[[0.298262, -0.8961559, 0.3285489], [0.4528000...",1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'who', 'type': 'say', 'tail': 'plant...","[{'head': 'Forest', 'type': 'plans', 'tail': '...",2,16,"[(Forest, plans, fires), (Forest, plans, tree)]","[(who, say, plants), (fire, say, forests), (bl...","{'(Forest, plans, fires)': ['(fire, say, fores...","[[-0.70220375, 0.71068317, -0.0428862], [-0.46...",1,1


In [6]:
def get_justification(row):
    claim_evidences = row['spacy_claim_evidence_1']
    entailment_scores = row['spacy_entailment_scores_1']

    claims = list(claim_evidences.keys())
    
    index_values = [-1, 1, 0]

    justifications = []

    for i in range(len(claims)):
        which_verdict = entailment_scores[i].argmax()
        if len(claim_evidences[claims[i]]) > 0:
            evidence = claim_evidences[claims[i]][0]
        else:
            evidence = ""
        claim = claims[i]
  
        justification = get_justification_statement(claim, evidence, index_values[which_verdict])

        justifications.append(justification)

    justification_mtl = ' '.join(justifications)
    
    row['justification_mtl'] = justification_mtl

    if row.name%100==0:
        print(row.name)
    
    return row


In [7]:
data_df = data_df.apply(lambda row : get_justification(row), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [8]:
data_df

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,spacy_claim_kg,spacy_claim_kg_size,spacy_main_text_kg_size,spacy_claim_triples,spacy_main_text_triples,spacy_claim_evidence_1,spacy_entailment_scores_1,verdict_actual,verdict,justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[{'head': 'baby', 'type': 'died', 'tail': 'fac...",2,54,"[(baby, died, facility), (nurse, died, child)]","[(@FierceFemtivist, published, series), (child...","{'(baby, died, facility)': ['(baby, reads, eve...","[[0.67679816, -0.60078514, -0.4254427], [0.677...",0,-1,The name of the facility where the event was h...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'Bat', 'type': 'tests', 'tail': 'Cou...",2,12,"[(Bat, tests, County), (Bat, tests, rabies)]","[(bat, reports, County), (Department, urging, ...","{'(Bat, tests, County)': ['(bat, reports, Coun...","[[-0.20893203, -0.7196208, 0.66218823], [0.569...",1,1,The reports for the game unrelated to bat were...
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'Germany', 'type': 'banned', 'tail':...",3,168,"[(Germany, banned, pork), (Germany, banned, ca...","[(cafes, reported, products), (GERMANY, made, ...","{'(Germany, banned, pork)': ['(frenzy, banned,...","[[-0.20289786, -0.7068694, 0.67761946], [0.638...",-1,-1,Pork is banned in Germany because it is unrela...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'Coronavirus', 'type': 'prompts', 't...",3,114,"[(Coronavirus, prompts, drugs), (Coronavirus, ...","[(government, urged, provinces), (government, ...","{'(Coronavirus, prompts, drugs)': ['(who, ramp...","[[0.298262, -0.8961559, 0.3285489], [0.4528000...",1,1,Coronavirus is unrelated to who. Coronavirus i...
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'Forest', 'type': 'plans', 'tail': '...",2,16,"[(Forest, plans, fires), (Forest, plans, tree)]","[(who, say, plants), (fire, say, forests), (bl...","{'(Forest, plans, fires)': ['(fire, say, fores...","[[-0.70220375, 0.71068317, -0.0428862], [-0.46...",1,1,"Because fire is a plan for forests, there are ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,3295,Christmas miracle for Florida dog whose heart ...,NaN,A Florida man was hoping for a Christmas mirac...,Associated Press,The heart of Gerald Ford’s 1-year-old French b...,https://www.gainesville.com/news/20200102/holi...,true,"Animals, Health, Florida, General News, Gaines...",[{'head': 'Christmas miracle for Florida dog w...,...,[],0,43,[],"[(heart, stopped, Neo), (heart, stopped, attac...",{},None,1,0,
1231,6904,Another dolphin dies at Arizona aquatic facility.,NaN,A fourth dolphin has 

In [9]:
print(data_df.iloc[5]['claim'])
print(data_df.iloc[5]['explanation'])
print(data_df.iloc[5]['label'])
print(data_df.iloc[5]['justification_mtl'])

School closures will have little impact on COVID-19 control, review finds.
School closures do not tend to help contain the spread of infections during outbreaks of disease such as COVID-19, but will have a big impact on how societies restart after lockdown, scientists said on Monday.
true
Because findings is what is the impact. Because closures is not finnds is control.


In [10]:
data_df.to_pickle('./data/spacy/data_justifications.pkl')

In [11]:
data_df = pd.read_pickle('./data/spacy/data_justifications.pkl')

In [12]:
bertscore = load("bertscore")
rouge = load('rouge')
sacrebleu = load('sacrebleu')

def get_metrics(predictions, references, references_sb):
    results = {}    
    results["bertscore"] = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    results["rouge"] = rouge.compute(predictions=predictions, references=references)
    results["sacrebleu"] = sacrebleu.compute(predictions=predictions, references=references_sb)

    return results

In [13]:
justifications_mtl = data_df['justification_mtl'].tolist()
references = data_df['explanation'].tolist()
references_sb = [[reference] for reference in references]

In [14]:
results_mtl = get_metrics(justifications_mtl, references, references_sb)
print(results_mtl)

{'bertscore': {'precision': [0.734406054019928, 0.7745344042778015, 0.7855608463287354, 0.7682191729545593, 0.7481989860534668, 0.7477642893791199, 0.721607506275177, 0.0, 0.0, 0.0, 0.0, 0.7909750938415527, 0.0, 0.714309573173523, 0.6931523084640503, 0.0, 0.7390863299369812, 0.7742388248443604, 0.7592419981956482, 0.7711397409439087, 0.8019735217094421, 0.0, 0.792411744594574, 0.7534381151199341, 0.7152199745178223, 0.7265936136245728, 0.823502242565155, 0.7449131011962891, 0.0, 0.70982825756073, 0.7938336133956909, 0.7821218967437744, 0.7688409090042114, 0.7247354984283447, 0.0, 0.0, 0.7543249130249023, 0.7190827131271362, 0.0, 0.7171206474304199, 0.7839264869689941, 0.0, 0.0, 0.7016556859016418, 0.7336545586585999, 0.7990444302558899, 0.0, 0.733902096748352, 0.7478708624839783, 0.8021289110183716, 0.7658935189247131, 0.7800135612487793, 0.7077562808990479, 0.7382787466049194, 0.7856888771057129, 0.7333904504776001, 0.7265012860298157, 0.8282305002212524, 0.7070436477661133, 0.7239470

In [23]:
mtl_precision = sum(results_mtl['bertscore']['precision']) / len(results_mtl['bertscore']['precision'])
mtl_recall = sum(results_mtl['bertscore']['recall']) / len(results_mtl['bertscore']['recall'])
mtl_f1score = sum(results_mtl['bertscore']['f1']) / len(results_mtl['bertscore']['f1'])

print(f"BERTScore Precision: {mtl_precision}, Recall: {mtl_recall}, F1-score: {mtl_f1score}")

BERTScore Precision: 0.6378432206481355, Recall: 0.590168115025049, F1-score: 0.6122704977944469


In [24]:
print(results_mtl['rouge'])

{'rouge1': 0.12394103993767025, 'rouge2': 0.010862979819433351, 'rougeL': 0.09206506107134355, 'rougeLsum': 0.09218885847482265}


In [25]:
print(results_mtl['sacrebleu'])

{'score': 0.03837888985277136, 'counts': [13446, 1237, 132, 34], 'totals': [47763, 45699, 43635, 41571], 'precisions': [28.151498021481064, 2.7068426004945403, 0.3025094534204194, 0.08178778475379471], 'bp': 0.03275249816276371, 'sys_len': 47763, 'ref_len': 211054}


In [18]:
predictions = ["My name is Prahlad, I study at Columbia"]
references = ["My university is Columbia, I am Prahlad"]
results = sacrebleu.compute(predictions=predictions, references=references, tokenize='none', lowercase=True, use_effective_order=True)
print(results)

{'score': 7.267884212102741, 'counts': [3, 0, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [37.5, 7.142857142857143, 4.166666666666667, 2.5], 'bp': 1.0, 'sys_len': 8, 'ref_len': 7}


In [19]:
def get_stats(data_df, columns):

    for column in columns:
        data_df[f'{column}_tokens'] = data_df[column].apply(lambda n: len(str(n).split()))

get_stats(data_df, columns=['claim', 'main_text', 'explanation'])

In [20]:
small_just_df = data_df[(data_df['main_text_tokens'] < 100) & (data_df['main_text_tokens'] > 10) ]

In [21]:
small_just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,spacy_claim_triples,spacy_main_text_triples,spacy_claim_evidence_1,spacy_entailment_scores_1,verdict_actual,verdict,justification_mtl,claim_tokens,main_text_tokens,explanation_tokens
36,5583,Alabama: Case of meningococcal disease in Jack...,NaN,The Alabama Department of Public Health’s Immu...,,"In a news release Tuesday, the department said...",,true,"Immunizations, Meningitis, Health, Alabama, Pu...","[{'head': 'Alabama', 'type': 'contains adminis...",...,[],"[(case, said, School), (case, said, County), (...",{},None,1,0,,8,90,34
121,40135,Little Lauren Renee Pingel was in a car accid...,"March 17, 2015",Pray for 3-year-old Lauren Renee Pingel in Ama...,Rich Buhler & Staff,This story is true… and the latest news is tha...,https://twitter.com/intent/tweet?text=Pray+for...,true,Prayers,"[{'head': 'Little Lauren Renee Pingel', 'type'...",...,"[(Pingel, was, accident), (Pingel, was, Amaril...","[(news, is, injuries), (news, is, treatment), ...","{'(Pingel, was, accident)': ['(Lauren, endured...","[[0.6981675, -0.5963186, -0.39618972], [0.6861...",1,-1,Lauren had swelling because of Pingel's accide...,32,69,9
189,7264,Zambia to vaccinate 1 million against cholera ...,NaN,Zambia is set to vaccinate one million people ...,,The World Health Organization says doses of th...,,true,"Health, Cholera, Zambia, Africa, Southern Afri...","[{'head': 'outbreak', 'type': 'has cause', 'ta...",...,[],"[(doses, says, vaccine), (doses, says, nation)...",{},None,1,0,,9,93,18
190,5834,State to labs: report all results of tests for...,NaN,Louisiana’s Department of Health says laborato...,,Louisiana has very high rates of all three dis...,http://ldh.la.gov/index.cfm/newsroom/detail/5162,true,"Health, Hepatitis, Syphilis, Louisiana, Public...","[{'head': 'tests', 'type': 'facet of', 'tail':...",...,[],"[(Louisiana, has, rates), (Louisiana, has, dis...",{},None,1,0,,11,96,27
237,4952,Health care worker in Grant County has mumps.,NaN,The Grant County Health District has issued an...,,The health district says a vaccinated health c...,http://www.kxly.com/,true,"Health, Moses Lake, Mumps",[{'head': 'Health care worker in Grant County'...,...,"[(worker, has, County), (worker, has, mumps)]","[(worker, says, mumps), (TV, reports, Spokane)...","{'(worker, has, County)': ['(district, working...","[[0.63329124, -0.67518353, -0.37824532], [-0.7...",1,1,Because district doesn't have county has staff...,8,84,20


In [27]:
which_just = small_just_df.iloc[35]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['spacy_claim_evidence_1'])
print(which_just['spacy_entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['justification_mtl'])

Nestle recalls batch of Alfamino infant formula in Germany.
“A baby that consumes a product from amongst these few (faulty) tins, can become sick with symptoms such as nausea, vomiting and headaches,” Nestle Germany said on its website late on Wednesday. “We urge you to immediately seek out a doctor in such cases.”  The faulty products were amongst Alfamino 400-gram tins produced under the batch number 80250346GA. Nestle warned parents not to give the formula produced under this batch to their children, adding faulty products could be recognized by their grey or green color after preparation.
Nestle has recalled a batch of its Alfamino amino acid specialist infant formula in Germany, saying a number of the products have a substantially increased dose of minerals that could make children sick.
true
[{'head': 'Alfamino', 'type': 'owned by', 'tail': 'Nestle', 'meta': {'spans': [[0, 128]]}}, {'head': 'Nestle', 'type': 'owner of', 'tail': 'Alfamino', 'meta': {'spans': [[0, 128]]}}, {'head':

# Data Source #4: LlaMa

In [30]:
data_df = pd.read_pickle('./data/llama/data_verdict.pkl')

In [31]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,llm_claim_kg,llm_main_text_kg,llama_claim_kg_size,llama_main_text_kg_size,llama_claim_triples,llama_main_text_triples,llama_claim_evidence_1,llama_entailment_scores_1,verdict_actual,verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[{'head': 'baby', 'type': 'died', 'tail': 'med...","[{'head': 'Confederate flag', 'type': 'instanc...",2,14,"[(baby, died, medical-facility), (medical-faci...","[(Confederate flag, instance of, racist), (Fie...","{'(baby, died, medical-facility)': ['(cardiac ...","[[-0.08809295, -0.7930231, 0.60278845], [0.649...",0,1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'Bat', 'type': 'from', 'tail': 'Shaw...","[{'head': 'KSNT', 'type': 'located in the admi...",2,6,"[(Bat, from, Shawnee County), (Shawnee County,...","[(KSNT, located in the administrative territor...","{'(Bat, from, Shawnee County)': ['(Topeka, loc...","[[0.6422565, -0.5995739, -0.477512], [-0.28233...",1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'Germany', 'type': 'has', 'tail': 'b...","[{'head': 'Express', 'type': 'country', 'tail'...",2,36,"[(Germany, has, banned), (pork, from, school c...","[(Express, country, British), (Express, instan...","{'(Germany, has, banned)': ['(Germany, contine...","[[0.51010597, -0.85914826, 0.04069696], [-0.26...",-1,1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'Coronavirus', 'type': 'prompts', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",2,21,"[(Coronavirus, prompts, Canada), (Canada, roll...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, prompts, Canada)': ['(coronavi...","[[-0.7723689, 0.030968107, 0.6344187], [-0.440...",1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'Wayne National Forest', 'type': 'pl...","[{'head': 'oak forests', 'type': 'located in t...",2,3,"[(Wayne National Forest, plans, fires), (tree,...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, plans, fires)': ['(o...","[[0.6409391, -0.7056944, -0.30198097], [-0.570...",1,1


In [33]:
def get_justification(row):
    claim_evidences = row['llama_claim_evidence_1']
    entailment_scores = row['llama_entailment_scores_1']

    claims = list(claim_evidences.keys())
    
    index_values = [-1, 1, 0]

    justifications = []

    for i in range(len(claims)):
        which_verdict = entailment_scores[i].argmax()
        if len(claim_evidences[claims[i]]) > 0:
            evidence = claim_evidences[claims[i]][0]
        else:
            evidence = ""
        claim = claims[i]
  
        justification = get_justification_statement(claim, evidence, index_values[which_verdict])

        justifications.append(justification)

    justification_mtl = ' '.join(justifications)
    
    row['justification_mtl'] = justification_mtl

    if row.name%100==0:
        print(row.name)
    
    return row


In [34]:
data_df = data_df.apply(lambda row : get_justification(row), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [35]:
data_df

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,llm_main_text_kg,llama_claim_kg_size,llama_main_text_kg_size,llama_claim_triples,llama_main_text_triples,llama_claim_evidence_1,llama_entailment_scores_1,verdict_actual,verdict,justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[{'head': 'Confederate flag', 'type': 'instanc...",2,14,"[(baby, died, medical-facility), (medical-faci...","[(Confederate flag, instance of, racist), (Fie...","{'(baby, died, medical-facility)': ['(cardiac ...","[[-0.08809295, -0.7930231, 0.60278845], [0.649...",0,1,A baby died in a medical-facility due to an un...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'KSNT', 'type': 'located in the admi...",2,6,"[(Bat, from, Shawnee County), (Shawnee County,...","[(KSNT, located in the administrative territor...","{'(Bat, from, Shawnee County)': ['(Topeka, loc...","[[0.6422565, -0.5995739, -0.477512], [-0.28233...",1,1,"Bat is not from Shawnee County, which is locat..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'Express', 'type': 'country', 'tail'...",2,36,"[(Germany, has, banned), (pork, from, school c...","[(Express, country, British), (Express, instan...","{'(Germany, has, banned)': ['(Germany, contine...","[[0.51010597, -0.85914826, 0.04069696], [-0.26...",-1,1,Germany is not banned in Europe. Pork is eaten...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'heroin', 'type': 'instance of', 'ta...",2,21,"[(Coronavirus, prompts, Canada), (Canada, roll...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, prompts, Canada)': ['(coronavi...","[[-0.7723689, 0.030968107, 0.6344187], [-0.440...",1,1,Coronavirus is an unrelated to coronavirus out...
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'oak forests', 'type': 'located in t...",2,3,"[(Wayne National Forest, plans, fires), (tree,...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, plans, fires)': ['(o...","[[0.6409391, -0.7056944, -0.30198097], [-0.570...",1,1,The Wayne National Forest is located in Ohio a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,3295,Christmas miracle for Florida dog whose heart ...,NaN,A Florida man was hoping for a Christmas mirac...,Associated Press,The heart of Gerald Ford’s 1-year-old French b...,https://www.gainesville.com/news/20200102/holi...,true,"Animals, Health, Florida, General News, Gaines...",[{'head': 'Christmas miracle for Florida dog w...,...,"[{'head': 'Neo', 'type': 'animal breed', 'tail...",2,12,"[(Christmas, miracle, Florida), (dog, heart, s...","[(Neo, animal breed, French bulldog), (Gerald

In [36]:
print(data_df.iloc[5]['claim'])
print(data_df.iloc[5]['explanation'])
print(data_df.iloc[5]['label'])
print(data_df.iloc[5]['justification_mtl'])

School closures will have little impact on COVID-19 control, review finds.
School closures do not tend to help contain the spread of infections during outbreaks of disease such as COVID-19, but will have a big impact on how societies restart after lockdown, scientists said on Monday.
true
The SARS outbreak happened in China and the school closed. The COVID-19 pandemic was caused by SARS-COV2 coronavirus. The COVID-19 pandemic was unrelated to SARS-COV2 coronavirus.


In [37]:
data_df.to_pickle('./data/llama/data_justifications.pkl')

In [38]:
data_df = pd.read_pickle('./data/llama/data_justifications.pkl')

In [39]:
bertscore = load("bertscore")
rouge = load('rouge')
sacrebleu = load('sacrebleu')

def get_metrics(predictions, references, references_sb):
    results = {}    
    results["bertscore"] = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    results["rouge"] = rouge.compute(predictions=predictions, references=references)
    results["sacrebleu"] = sacrebleu.compute(predictions=predictions, references=references_sb)

    return results

In [40]:
justifications_mtl = data_df['justification_mtl'].tolist()
references = data_df['explanation'].tolist()
references_sb = [[reference] for reference in references]

In [41]:
results_mtl = get_metrics(justifications_mtl, references, references_sb)
print(results_mtl)

{'bertscore': {'precision': [0.7098424434661865, 0.7141662836074829, 0.7912895679473877, 0.7883642315864563, 0.7794357538223267, 0.7537153959274292, 0.7496472597122192, 0.8608064651489258, 0.7999600172042847, 0.736111044883728, 0.8054811954498291, 0.8383390307426453, 0.8409250974655151, 0.7448925375938416, 0.7137505412101746, 0.5796453952789307, 0.7937414646148682, 0.7788016200065613, 0.7379884719848633, 0.8084202408790588, 0.7884459495544434, 0.801181435585022, 0.8220164775848389, 0.7995270490646362, 0.7091681957244873, 0.761022686958313, 0.7917701005935669, 0.7111004590988159, 0.8138695359230042, 0.7864667177200317, 0.7478523254394531, 0.844598650932312, 0.8207104802131653, 0.7615305185317993, 0.7660126686096191, 0.7652378678321838, 0.7643283605575562, 0.7273727059364319, 0.7559506297111511, 0.759318470954895, 0.7297645807266235, 0.7989025115966797, 0.7823752164840698, 0.7781088352203369, 0.7667505741119385, 0.8264709115028381, 0.8111162781715393, 0.750621497631073, 0.765417993068695

In [42]:
mtl_precision = sum(results_mtl['bertscore']['precision']) / len(results_mtl['bertscore']['precision'])
mtl_recall = sum(results_mtl['bertscore']['recall']) / len(results_mtl['bertscore']['recall'])
mtl_f1score = sum(results_mtl['bertscore']['f1']) / len(results_mtl['bertscore']['f1'])

print(f"BERTScore Precision: {mtl_precision}, Recall: {mtl_recall}, F1-score: {mtl_f1score}")

BERTScore Precision: 0.772799804126069, Recall: 0.7310317535238982, F1-score: 0.7501559526846421


In [43]:
print(results_mtl['rouge'])

{'rouge1': 0.20037026037812533, 'rouge2': 0.04563264953539377, 'rougeL': 0.14449923746439158, 'rougeLsum': 0.14449086276230838}


In [44]:
print(results_mtl['sacrebleu'])

{'score': 0.6634781052886836, 'counts': [23649, 4268, 1309, 554], 'totals': [74653, 72201, 69749, 67297], 'precisions': [31.678566166128622, 5.911275467098793, 1.8767294154754908, 0.8232164881049675], 'bp': 0.1608740468555779, 'sys_len': 74653, 'ref_len': 211054}


In [45]:
predictions = ["My name is Prahlad, I study at Columbia"]
references = ["My university is Columbia, I am Prahlad"]
results = sacrebleu.compute(predictions=predictions, references=references, tokenize='none', lowercase=True, use_effective_order=True)
print(results)

{'score': 7.267884212102741, 'counts': [3, 0, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [37.5, 7.142857142857143, 4.166666666666667, 2.5], 'bp': 1.0, 'sys_len': 8, 'ref_len': 7}


In [46]:
def get_stats(data_df, columns):

    for column in columns:
        data_df[f'{column}_tokens'] = data_df[column].apply(lambda n: len(str(n).split()))

get_stats(data_df, columns=['claim', 'main_text', 'explanation'])

In [47]:
small_just_df = data_df[(data_df['main_text_tokens'] < 100) & (data_df['main_text_tokens'] > 10) ]

In [48]:
small_just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,llama_claim_triples,llama_main_text_triples,llama_claim_evidence_1,llama_entailment_scores_1,verdict_actual,verdict,justification_mtl,claim_tokens,main_text_tokens,explanation_tokens
36,5583,Alabama: Case of meningococcal disease in Jack...,NaN,The Alabama Department of Public Health’s Immu...,,"In a news release Tuesday, the department said...",,true,"Immunizations, Meningitis, Health, Alabama, Pu...","[{'head': 'Alabama', 'type': 'contains adminis...",...,"[(Alabama, case, meningococcal), (Jackson Coun...","[(rash, subclass of, symptoms), (rash, instanc...","{'(Alabama, case, meningococcal)': ['(rash, su...","[[0.6673981, -0.6386944, -0.3829482], [-0.7601...",1,1,Alabama does not have meningococcal because ra...,8,90,34
121,40135,Little Lauren Renee Pingel was in a car accid...,"March 17, 2015",Pray for 3-year-old Lauren Renee Pingel in Ama...,Rich Buhler & Staff,This story is true… and the latest news is tha...,https://twitter.com/intent/tweet?text=Pray+for...,true,Prayers,"[{'head': 'Little Lauren Renee Pingel', 'type'...",...,"[(Little Lauren Renee Pingel, wasInACarAcciden...","[(Lauren, date of birth, December 15, 2000), (...","{'(Little Lauren Renee Pingel, wasInACarAccide...","[[-0.46334302, -0.5047961, 0.7283503], [0.6700...",1,1,Little Lauren Renee Pingel was born on Decembe...,32,69,9
189,7264,Zambia to vaccinate 1 million against cholera ...,NaN,Zambia is set to vaccinate one million people ...,,The World Health Organization says doses of th...,,true,"Health, Cholera, Zambia, Africa, Southern Afri...","[{'head': 'outbreak', 'type': 'has cause', 'ta...",...,"[(Zambia, vaccinate, 1 million), (cholera, out...","[(Cholera, subclass of, acute diarrheal), (Lus...","{'(Zambia, vaccinate, 1 million)': ['(Zambia, ...","[[0.5698025, -0.817226, -0.086410485], [0.2064...",1,1,"Lusaka is the capital of Zambia, which does no...",9,93,18
190,5834,State to labs: report all results of tests for...,NaN,Louisiana’s Department of Health says laborato...,,Louisiana has very high rates of all three dis...,http://ldh.la.gov/index.cfm/newsroom/detail/5162,true,"Health, Hepatitis, Syphilis, Louisiana, Public...","[{'head': 'tests', 'type': 'facet of', 'tail':...",...,"[(State, to, labs), (labs, report, all)]","[(Centers for Disease Control and Prevention, ...","{'(State, to, labs)': ['(Centers for Disease C...","[[0.6297443, -0.68902326, -0.3587047], [-0.464...",1,1,The Centers for Disease Control and Prevention...,11,96,27
237,4952,Health care worker in Grant County has mumps.,NaN,The Grant County Health District has issued an...,,The health district says a vaccinated health c...,http://www.kxly.com/,true,"Health, Moses Lake, Mumps",[{'head': 'Health care worker in Grant County'...,...,"[(healthcareworkerinGrantCounty, has, mumps), ...","[(KXLY-TV, headquarters location, Spokane), (K...","{'(healthcareworkerinGrantCounty, has, mumps)'...","[[0.69396675, -0.6059742, -0.38885146], [0.642...",1,-1,HealthcareworkerinGrantCounty does not have mu...,8,84,20


In [ ]:
which_just = small_just_df.iloc[34]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['justification_mtl'])

Ohio officials say tick-borne Lyme Disease still a threat.
The state agency reports that 241 cases of Lyme disease have been reported in 60 of Ohio’s 88 counties this year. The number of cases has risen steadily the last five years, with 160 reported last year after only 93 in 2013. Lyme disease is passed to humans by deer ticks and can cause flu-like illnesses, muscle pain and headaches. A rash shaped like a bull’s-eye is often seen around tick bites. The Ohio Department of Natural Resources recommends people walk in the middle of trails and tuck in clothing to prevent bites.
The Ohio Department of Health says hunters and those trekking in wooded areas should use bug spray and take other precautions even in cold weather to prevent being infected with tick-borne Lyme disease.
true
[{'head': 'Lyme Disease', 'type': 'has cause', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'head': 'Lyme Disease', 'type': 'subclass of', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'hea

In [ ]:
which_just = small_just_df.loc[189]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['justification_mtl'])

Zambia to vaccinate 1 million against cholera amid outbreak.
The World Health Organization says doses of the cholera vaccine have been delivered to the impoverished southern African nation as fears spread in Lusaka. Zambia’s health ministry says 58 of the 63 countrywide deaths in the cholera outbreak that began in early October have occurred in the capital. Of the 2,672 cases across the country, 2,558 have been in Lusaka. Cholera is an acute diarrheal and bacterial infection caused by ingestion of contaminated food or water. Just hours after being infected, vomiting and diarrhea cause severe dehydration that can kill without rapid intervention.
Zambia is set to vaccinate one million people in its capital against cholera as a deadly outbreak grows.
true
[{'head': 'outbreak', 'type': 'has cause', 'tail': 'cholera', 'meta': {'spans': [[0, 128]]}}, {'head': 'outbreak', 'type': 'country', 'tail': 'Zambia', 'meta': {'spans': [[0, 128]]}}, {'head': 'vaccine', 'type': 'country', 'tail': 'Zambi